In [ ]:
#Import Data
from collections import defaultdict
import pandas as pd 
import numpy as np
import itertools    
from functools import reduce

!pip install openpyxl






: 

In [ ]:
Data = pd.read_excel('../input/vertical-d/vertical.xlsx')

: 

In [ ]:
#check if data is vertical
column_names = Data.columns.tolist()
for column in column_names:
    print(column) 
first_column=column_names[0]
print('first_column=',first_column)
sec_column=column_names[1]
print('sec_column=',sec_column) 

#dont forget to check the col names
if first_column == 'TiD' and sec_column=='items':
      lets_go=1
        
else:
    #dont forget to check the col names
    data = Data.set_index('item')['tid'].to_dict()

    horizontal_data = {}


    for item, tid in data.items():
        tids = str(tid).split(',')
        for t in tids:
            if t in horizontal_data:
                  horizontal_data[t].append(item)
            else:
                  horizontal_data[t] = [item]

    for col in horizontal_data:
        
        horizontal_data[col] = str(horizontal_data[col]).replace("[" , '')
        horizontal_data[col] = str(horizontal_data[col]).replace("]" , '')
        horizontal_data[col] = str(horizontal_data[col]).replace(" " , '')
        horizontal_data[col] = str(horizontal_data[col]).replace("'' " , '')
        print(horizontal_data[col])

    Data = pd.DataFrame(list(horizontal_data.items()), columns=['Key', 'items'])


: 

In [ ]:
print(Data)

: 

In [ ]:
#Variables
min_support = 3
min_conf = 0.3

: 

In [ ]:
#Helper Functions
def get_value(item, param):
    values = []
    value = 0
    for i in param[:-1]:
        for path in conditional_trees[item]:
            if(i == path):
                value = value + conditional_trees[item][path]
        values.append(value)
        value = 0
    return min(values)

def calculate_supports(combinations, item):  
    Supports = dict()
    Confidence = dict()
    transactions = Data['items'].str.split(',')
    item = str(item).split(",")
    for combo in combinations:
        support_count = 0
        item_count = 0
        for transaction in transactions:
            transaction = set(transaction)
            if(set(combo) <= transaction and set(item) <= transaction):
                support_count += 1
                
            if(set(item) <= transaction):
                item_count +=1

        key = ''.join(combo) + "--->" + str(item)
        if(len(transactions)>0):
            Supports[key] = support_count/len(transactions)
        if(item_count>0):
            Confidence[key] = support_count/ item_count
            
        key = ''.join(str(item)) + "--->" + ''.join(combo)
        if(len(transactions)>0):
            Supports[key] = support_count/len(transactions)
        if(support_count >0):   
            Confidence[key] = item_count / support_count
                
    return Supports, Confidence

: 

In [ ]:
def generate_frequent_itemset(df, min_support):

    #Split items in the lists
    transactions = df['items'].str.split(',')
    

    #Store itemsets and support count in dictionary
    item_counts = defaultdict(int)
    total_transactions = len(transactions)

    #Loop on transactions to count item occurrences
    for transaction in transactions:
        unique_items = set(transaction)
        for item in unique_items:
            item_counts[item] += 1


   # --> Check if min_support is a number or a percentage

     #if min_support is a number
    if isinstance(min_support, (int, float)) and not (0 < min_support <= 1):
        min_support_count = min_support

     #if min_support is a ratio/fraction
    elif isinstance(min_support, (int, float)) and (0 < min_support <= 1):
        min_support_count = min_support * total_transactions

     #if min_support is a percentage
    else:
        min_support_count = min_support * total_transactions / 100


    #calculate support and filter frequent items
    frequent_itemsets = {item: support for item, support in item_counts.items() if support >= min_support_count}
    print("Frequent Itemsets With Support >=" , min_support , ":")
    return frequent_itemsets

: 

In [ ]:
frequent_itemsets = generate_frequent_itemset(Data, min_support)
frequent_itemsets = dict(sorted(frequent_itemsets.items(), key=lambda x:(-x[1],x[0])))
print(frequent_itemsets)

: 

In [ ]:
def get_sorted_transactions(frequent_itemsets):
    transactions = Data['items'].str.split(',')

    T1 = defaultdict(int) #for each transaction
    T2  = []              #to copy values into
    Transactions = []     #array of transactions

    for transaction in transactions:
        items = set(transaction)
        for item in items:
            if frequent_itemsets.get(item) is not None:
                T1.update({item : frequent_itemsets[item]})
        T2 = T1.copy()
        Transactions.append(T2)
        T1.clear()

    Sorted_Transactions = []
    for transaction in Transactions:
        sorted_transactions = sorted(transaction.items(), key=lambda x:(-x[1],x[0]))
        Sorted_Transactions.append(dict(sorted_transactions))

    return Sorted_Transactions

: 

In [ ]:
sorted_transactions = get_sorted_transactions(frequent_itemsets)
print(sorted_transactions)

: 

In [ ]:
def get_conditional_pattern_base(frequent_itemsets,sorted_sransactions):

    x = dict(sorted(frequent_itemsets.items(), key=lambda x:(-x[1],x[0])))
    sorted_frequent_itemsets = x.keys() #menna handled this already?
    pattern_base = defaultdict(int)
    temp = defaultdict(int) #used to copy
    coditional_pattern_base = dict()


    for item in sorted_frequent_itemsets:
        for i in range(len(sorted_sransactions)):
            if (sorted_sransactions[i].get(item) is not None):
                transaction = sorted_sransactions[i].keys()
                index = list(transaction).index(item)
                path1 = list(transaction)[:index]
                path = ''
                if(len(path1) > 0):
                    for i in range(len(path1)):
                        path+=path1[i] + ','
                    path = list(path)
                    path.pop()
                    path = ''.join(path)
                pattern_base[path] += 1

        temp = pattern_base.copy()
        coditional_pattern_base[item] = temp
        if('' in coditional_pattern_base[item]):
            coditional_pattern_base[item].pop('')
        pattern_base.clear()
    
    return coditional_pattern_base; 

: 

In [ ]:
coditional_pattern_base = get_conditional_pattern_base(frequent_itemsets,sorted_transactions)
frequent_itemsets = dict(sorted(frequent_itemsets.items(), key=lambda x:(-x[1],x[0]), reverse = True))
for item in frequent_itemsets.keys():
    if(dict(coditional_pattern_base[item])):
        print("item:" , item, dict(coditional_pattern_base[item]))

: 

In [ ]:
def get_conditional_trees(coditional_pattern_base):
    conditional_trees = dict()
    x = defaultdict(int)
    common_list = {}

    for item in coditional_pattern_base.keys():
        summ = sum(coditional_pattern_base[item].values())
        #if list is not empty
        if(coditional_pattern_base[item]):
            common = list(coditional_pattern_base[item])[0]

            common = str(common).split(',')

            for path in coditional_pattern_base[item]:
                path = str(path).split(',')
                common = sorted(set(common) & set(path) , key= common.index)
       

            for letter in common:
                common_list[letter] = summ
            temp = common_list.copy()
            conditional_trees[item] = temp
            common_list.clear()

    return conditional_trees


: 

In [ ]:
conditional_trees = get_conditional_trees(coditional_pattern_base)
for item in conditional_trees:
    print(item, dict(conditional_trees[item]))

: 

In [ ]:
def get_frequent_pattern(conditional_trees):
    letters_combo = []
    temp = []
    temp2 = []
    frequent_pattern = {}
    x = {}
    for item in conditional_trees:
        trees_list = dict(conditional_trees[item])

        #items to make combinations of
        for letter in trees_list:
            letters_combo.append(letter)
        letters_combo.append(item)


        for i in range(1, len(letters_combo)+1):
            combinations = list((itertools.combinations(letters_combo, i+1)))

            #remove wrong combos
            for combo in combinations:
                combosrt = str(combo)
                if(str(combosrt).find(str(item)) == -1):
                    combinations.remove(combo)  

            temp = temp+ combinations.copy() 
        temp2 = temp.copy()
        for i in temp2:
            x[i] = get_value(item, i);
        temp3 = x.copy()
        frequent_pattern[item] = temp3
        letters_combo.clear()
        temp.clear()
        x.clear()
    return frequent_pattern



: 

In [ ]:
frequent_pattern = get_frequent_pattern(conditional_trees)
for item in frequent_pattern:
    print(item, frequent_pattern[item])
   

: 

In [ ]:
def calculate_lift(df):
    items = df['items'].str.split(',')
    total_items = len(df)
    
    unique_items = list({item for sublist in items for item in sublist})

    item_probabilities = {}
    for item in unique_items:
        item_probabilities[item] = df[df['items'].str.contains(item)].shape[0] / total_items

    lift_output = pd.DataFrame(columns=['first_item', 'second_item', 'Lift', 'Dependency & Correlation'])

    for i in range(len(unique_items)):
        for j in range(i+1, len(unique_items)):
            first_item = unique_items[i]
            second_item = unique_items[j]

            first_item_probability = item_probabilities[first_item]
            second_item_probability = item_probabilities[second_item]
            probability_items_union = df[df['items'].str.contains(first_item) & df['items'].str.contains(second_item)].shape[0] / total_items

            if first_item_probability == 0 or second_item_probability == 0:
                lift = "Can't calculate lift: Division by zero"
            else:
                lift = probability_items_union / (first_item_probability * second_item_probability)
                dependency_correlation = "dependent, +ve correlated" if lift > 1 else "dependent, -ve correlated" if lift < 1 else "independent"

            new_row = {'first_item': first_item, 'second_item': second_item, 'Lift': lift, 'Dependency & Correlation': dependency_correlation}
            lift_output = pd.concat([lift_output, pd.DataFrame([new_row])], ignore_index=True)

    return lift_output


: 

In [ ]:
lift_output = calculate_lift(Data)
print(lift_output)

: 

In [ ]:
itemsets = generate_frequent_itemset(Data, min_support)
itemsets = dict(sorted(itemsets.items(), key=lambda x:(-x[1],x[0])))
print(itemsets)

: 

In [ ]:
temp = []
print("Rule                           Support                    Confidence ")
print("====================================================================")
for item in itemsets:
    #print(items)
    items = list(itemsets.keys())
    for i in range(1, len(items)):
        combinations = list((itertools.combinations(items, i)))
        temp = temp + combinations

    Supports, Confidence = calculate_supports(temp, item)
    for key in Confidence:
        print("%-30s %-30f %-30s " %(key, Supports[key], Confidence[key]))
    combinations.clear()
    temp.clear()
    


: 

In [ ]:
temp = []
print ("                             Strong Rules                     ")
print("Rule                           Support                    Confidence ")
print("====================================================================")
for item in itemsets:
    #print(items)
    items = list(itemsets.keys())
    items.remove(item)
    for i in range(1, len(items)):
        combinations = list((itertools.combinations(items, i)))
        temp = temp + combinations
        
    Supports, Confidence = calculate_supports(temp, item)
    for key in Supports:
        if(Supports[key] >= min_support and Confidence[key]>= min_conf):
            print("%-30s %-30f %-30s " %(key, Supports[key], Confidence[key]))
    combinations.clear()
    temp.clear()
    


: 